#### Training a XGBoost Classifier

Our approach was to develop a XGBClassifier to predict the "NEXT ACTION"

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn import SKLearn
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
from datetime import datetime
import pytz
import ast

from sagemaker.local import LocalSession

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [9]:
role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()

# Local Session
#sess = LocalSession()
#sess.config = {'local': {'local_code':True}}

Bucket = 'sagemaker-us-east-1-254050731868'
Prefix = 'starbucks-capstone'

In [3]:
train_path = f's3://{Bucket}/{Prefix}/dataset/train.csv'
test_path = f's3://{Bucket}/{Prefix}/dataset/test.csv'

#### Preparing the estimator for the hyperparameter tuner job

In [15]:
estimator = Estimator(entry_point='code/train.py',
                      image_uri='683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3',
                      instance_count=1,
                      instance_type='ml.m5.large',
                      #instance_type='local',
                      framework_version='1.2-1',
                      code_location=f's3://{Bucket}/{Prefix}/model',
                      ouptut_path=f's3://{Bucket}/{Prefix}/model',
                      dependencies=['code/requirements.txt'],
                      role=role,
                      max_run=3600,
                     )

params = {
    "max-depth":6,         # 0-inf (int)
    "min-child-weight": 1, # 0-inf (int)
    "subsample":1,         # 0-1 (float)
    "colsample-bytree":1,  # 0-1 (float)
    "eta": 0.3,            # 0-1 (float)
}

estimator.set_hyperparameters(**params)

#### Launch the hyperparameter tuner job

In [16]:
now = datetime.now()
now_suffix = now.strftime('%b%d%y-%H%M%S').lower()

tuner_args = dict(
    objective_metric_name='test_precision',
    objective_type='Maximize',
    metric_definitions=[{'Name':'test_precision', 'Regex': 'TEST  PRECISION: (-?[0-9\.]+)'}],
    max_jobs=12,
    max_parallel_jobs=3,
    early_stopping_type='Auto'
)

tuner = HyperparameterTuner(
    estimator=estimator,
    hyperparameter_ranges={
        "max-depth"        : IntegerParameter(2, 400),          # 0-inf (int)
        "min-child-weight" : IntegerParameter(1, 400),          # 0-inf (int)
        "subsample"        : ContinuousParameter(0.0001, 0.99), # 0-1 (float)
        "colsample-bytree" : ContinuousParameter(0.0001, 0.99), # 0-1 (float)
        "eta"              : ContinuousParameter(0.0001, 0.99)  # 0-1 (float)
    },
    strategy='Bayesian',
    max_runtime_in_seconds=4*3600,
    base_tuning_job_name='starbucks',
    **tuner_args
)

tuner.fit(
    inputs={'train': train_path, 'test': test_path},
    wait=True,
    job_name=f'tn-stb-event-{now_suffix}'
)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


..........................................................................................................................!


#### Optionally you can train the final model

We prefer this last part to be tested on local to prevent failures of the hyperparameter tunning job rather than training a final model.

We could get the final model from our best hyperparameter tunning job.

In [41]:
now = datetime.now()
now_suffix = now.strftime('%b%d%y-%H%M%S').lower()

estimator.fit(
    wait = True,
    job_name=f'starbucks-{now_suffix}',
    inputs={"train":train_path, "test":test_path}
)

INFO:sagemaker:Creating training-job with name: starbucks-sep2124-192622
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.image:docker compose f

 Container lmx9ecta00-algo-1-gn0q7  Creating
 Container lmx9ecta00-algo-1-gn0q7  Created
Attaching to lmx9ecta00-algo-1-gn0q7
lmx9ecta00-algo-1-gn0q7  | 2024-09-21 19:26:24,729 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
lmx9ecta00-algo-1-gn0q7  | 2024-09-21 19:26:24,733 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
lmx9ecta00-algo-1-gn0q7  | 2024-09-21 19:26:24,736 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
lmx9ecta00-algo-1-gn0q7  | 2024-09-21 19:26:24,748 sagemaker-training-toolkit INFO     instance_groups entry not present in resource_config
lmx9ecta00-algo-1-gn0q7  | 2024-09-21 19:26:24,755 sagemaker_sklearn_container.training INFO     Invoking user training script.
lmx9ecta00-algo-1-gn0q7  | 2024-09-21 19:26:25,140 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
lmx9ecta00-algo-1-gn0q7  | /miniconda3/bin/python -m pip inst

INFO:root:creating /tmp/tmpot49xiw7/artifacts/output/data
INFO:root:copying /tmp/tmpot49xiw7/algo-1-gn0q7/output/success -> /tmp/tmpot49xiw7/artifacts/output
INFO:root:copying /tmp/tmpot49xiw7/model/model.joblib -> /tmp/tmpot49xiw7/artifacts/model


lmx9ecta00-algo-1-gn0q7 exited with code 0
Aborting on container exit...
 Container lmx9ecta00-algo-1-gn0q7  Stopping
 Container lmx9ecta00-algo-1-gn0q7  Stopped


INFO:sagemaker.local.image:===== Job Complete =====
